# Softmax Classification - Iris Data set

Softmax Classification의 Exercise를 찾던 중 붓꽃(Iris) 데이터셋을 이용하여 여러 개의 sigmoid 함수를 써서 다변수 classification을 해보는 실험을 찾았다.

참고 : <https://alphago.pe.kr/entry/4-TensorFlow%EC%99%80-%EB%86%80%EC%9E%90-Softmax-Classification>

아래 실험은 위의 참고 사이트의 실험을 그대로 카피한 소스코드이다.


In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

keys = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
data = pd.read_csv("iris.csv")
data = data.drop('caseno', axis=1)

species = list(data['Species'].unique())
data['class'] = data['Species'].map(lambda x: np.eye(len(species))[species.index(x)])

testset = data.sample(50)
trainset = data.drop(testset.index)

nb_classes = len(species)

X = tf.placeholder(tf.float32, [None,len(keys)])
Y = tf.placeholder(tf.float32, [None,len(species)])

W = tf.Variable(tf.zeros([len(keys),len(species)]), name= "Weight")
b = tf.Variable(tf.zeros([len(species)]), name= "Bias")

H = tf.nn.softmax(tf.matmul(X, W)+b)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=H, labels=Y, name="Cross_Entropy"))
optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(H,1),tf.argmax(Y,1)), tf.float32))

init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

trainset_class = [y for y in trainset['class'].values]

for step in xrange(1001):
    
    sess.run(optimizer, feed_dict={X: trainset[keys].values,
                                  Y: trainset_class})
    if step%100 == 0:
        loss, acc = sess.run([cross_entropy, accuracy], feed_dict={X: trainset[keys].values,
                                  Y: trainset_class})
        print "Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc)

accu =  sess.run(accuracy, feed_dict={X: testset[keys].values,
                                   Y: [y for y in testset['class'].values]})

print "\nAccuracy: {:.2%}\n".format(accu)
species = ["setosa", "versicolor", "virginica"]

sample = data.sample(1)
result = species[np.argmax(sess.run(H, feed_dict = {X: sample[keys].values}))]

print "The flower which has", sample[keys].values, "may be", result

if result == sample['Species'].values:
    print "Correct!"

else:
    print "Incorrect!"



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Step:     0	Loss: 1.083	Acc: 35.00%
Step:   100	Loss: 0.815	Acc: 69.00%
Step:   200	Loss: 0.765	Acc: 92.00%
Step:   300	Loss: 0.733	Acc: 98.00%
Step:   400	Loss: 0.710	Acc: 99.00%
Step:   500	Loss: 0.693	Acc: 99.00%
Step:   600	Loss: 0.679	Acc: 99.00%
Step:   700	Loss: 0.669	Acc: 99.00%
Step:   800	Loss: 0.660	Acc: 99.00%
Step:   900	Loss: 0.653	Acc: 99.00%
Step:  1000	Loss: 0.647	Acc: 99.00%

Accuracy: 92.00%

The flower which has [[5.6 2.7 4.2 1.3]] may be versicolor
Correct!
